In [ ]:
import pandas as pd
import numpy as np
# Set the option to display wrapped text
pd.set_option('display.max_colwidth', None)

I have the following DataFrame:

In [ ]:
my_dataframe = {
    'database': ['db_1', 'db_1', 'db_1', 'db_1'],
    'schema': ['schema_1', 'schema_1', 'schema_1', 'schema_1'],
    'table': ['table_1', 'table_1', 'table_1', 'table_1'],
    'column': ['column_1', 'column_2', 'column_3', 'column_4'],
    'th_sum': [
        {
            'result': 'error',
            'reason': 'No matching pattern 1!',
            'errror_while_parsing': [1,2,3]
        },
        np.NaN,
        {
            'result': 'error',
            'reason': 'No matching pattern 2!',
            'errror_while_parsing': [1,2,3]
        },
        np.NaN
    ],
    'th_null':  [
        np.NaN,
        {
            'result': 'error',
            'reason': 'No matching pattern 3!',
            'errror_while_parsing': [1,2,3]
        },
        {
            'result': 'error',
            'reason': 'No matching pattern 4!',
            'errror_while_parsing': [1,2,3]
        },
        np.NaN
    ],
    'th_max':  [
        np.NaN,
        {
            'result': 'error',
            'reason': 'No matching pattern 5!',
            'errror_while_parsing': [1,2,3]
        },
        {
            'result': 'error',
            'reason': 'No matching pattern 6!',
            'errror_while_parsing': [1,2,3]
        },
        {
            'result': 'error',
            'reason': 'No matching pattern 7!',
            'errror_while_parsing': [1,2,3]
        },
    ]
}
my_dataframe = pd.DataFrame(my_dataframe)
th_columns = ['th_sum', 'th_null', 'th_max']
my_dataframe

In [ ]:
def add_column_name_info(row):
    for col_name in th_columns:
        if isinstance(row[col_name], dict):
            row[col_name]['threshold_where_error_occur'] = col_name
    return row

In [ ]:
# Apply the custom function to each row
df_updated = my_dataframe.apply(add_column_name_info, axis=1)
df_updated

### Concatenate all threshold into one column

In [ ]:
# Creating a new column with a list of non-null values
df_updated['description'] = df_updated[th_columns].apply(lambda row: row[row.notna()].tolist(), axis=1)
df_updated = df_updated.drop(columns=th_columns).explode('description').reset_index(drop=True)
# Define the keys to extract
# keys_to_extract = ['result', 'reason']

# # Use apply and pd.Series to create new columns for each key
# df_updated[keys_to_extract] = df_updated['description'].apply(lambda x: pd.Series({key: x.get(key) for key in keys_to_extract}))

# # Drop the original 'description' column if needed
# df = df.drop(columns=['description'])
df_updated


In [ ]:
# Define the keys to extract
keys_to_extract = ['result', 'reason']
# Use apply and pd.Series to create new columns for each key and remove the keys from the original dictionaries
df_updated[keys_to_extract] = df_updated['description'].apply(lambda x: pd.Series({key: x.pop(key, None) for key in keys_to_extract}))
# Rename column
df_updated = df_updated.rename(columns={'description': 'additional information'})

# Reorder column
# Define the desired column order
desired_order = ['database', 'schema', 'table', 'column'] + keys_to_extract + ['additional information']

# Reorder columns
df_updated = df_updated[desired_order]

# Display the resulting DataFrame
df_updated




### Simulate error in database/schema/table/column and concatenate all errors together

In [ ]:
import pandas as pd
import numpy as np

dataframe_with_nulls = {
    'database': [np.NaN, np.NaN, 'db_1', 'db_1', np.NaN],
    'schema': [np.NaN, np.NaN, 'schema_1', 'schema_1', np.NaN],
    'table': [np.NaN, 'table_1', 'table_1', 'table_2', np.NaN],
    'column': [np.NaN, 'column_2', np.NaN, 'column_4', np.NaN]
}

df_common = pd.DataFrame(dataframe_with_nulls)
empty_condition = df_common[['database','schema','table','column']].isna().any(axis=1)
df_common = df_common[empty_condition]
# Add a 'reason' column with a list of values indicating where NaN values occur
df_common['description'] = df_common.apply(lambda row: [{'result':'error', 'reason': f'Empty entry found for: {col}'} for col in row.index[row.isna()]], axis=1)
df_common = df_common.explode('description')
# df_common[['result', 'reason']] = df_common['description'].apply(lambda x: pd.Series(x))
# df_common = pd.concat([df_common.drop('description', axis=1), df_common['description'].apply(lambda x: pd.Series(x))], axis=1)
df_common = pd.concat([df_common.drop('description', axis=1), 
                      df_common['description'].apply(lambda x: pd.Series(x)),
                      pd.DataFrame(columns=['additional information'])], axis=1)



# Display the resulting DataFrame
df_common


In [ ]:
# Assuming df_updated and df_common are your two DataFrames
df_combined = pd.concat([df_updated, df_common], axis=0, sort=False)
df_combined

